# Load Libraries

In [11]:
# basic
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

# for reading the data
from nbodykit.source.catalog import BigFileCatalog
from nbodykit.transform import ConcatenateSources, CartesianToEquatorial

In [16]:
sys.path.append('/lustre/work/akira.tokiwa/LensingSSC')

from lib.mpl_aea.mpl_aea import healpix

# Load Data

In [4]:
# the data directory
data_dir = '/lustre/work/akira.tokiwa/supersample/'
sample_path = data_dir + 'rfof_proc128_nc1024_size5000_nsteps60lin_ldr0_rcvnil_fstnone_pnf2_lnf2_s100_dhf1.0000_tiled0.20_fll_elllim_10000_npix_4096_rfofkdt_8/'

In [5]:
# read the data
rfof = BigFileCatalog(sample_path + "rfof_1.0000", dataset='RFOF/', header='header')
hpix = BigFileCatalog(sample_path + "usmesh", dataset='HEALPIX/', header='header')
meshs = BigFileCatalog(sample_path + "usmesh", dataset='1/', header='header')
mesh_rfof = BigFileCatalog(sample_path + "usmesh", dataset='RFOF/', header='header')

/home/akira.tokiwa/.conda/envs/cfastpm/lib/python3.8/site-packages/bigfile/__init__.py:358: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return pyxbigfile.Dataset.__init__(self, file, dtype=dtype, size=size)


In [9]:
# print the header
print("Read RFOF, columns = ", rfof.columns)
print("Read HEALPIX, columns = ", hpix.columns)
print("Read mesh, columns = ", meshs.columns)
print("Read mesh_rfof, columns = ", mesh_rfof.columns)

Read RFOF, columns =  ['ID', 'InitialPosition', 'Length', 'MinID', 'Position', 'RVdisp', 'Rdisp', 'Selection', 'Task', 'Value', 'Vdisp', 'Velocity', 'Weight']
Read HEALPIX, columns =  ['Aemit', 'ID', 'Mass', 'Rmom', 'Selection', 'Value', 'Weight']
Read mesh, columns =  ['Aemit', 'ID', 'Position', 'Selection', 'Value', 'Velocity', 'Weight']
Read mesh_rfof, columns =  ['Aemit', 'ID', 'Length', 'MinID', 'Position', 'RVdisp', 'Rdisp', 'Selection', 'Task', 'Value', 'Vdisp', 'Velocity', 'Weight']


# Functions

In [10]:
def read_range(cat, amin, amax):
    """ Read a portion of the lightcone between two red shift ranges
        The lightcone from FastPM is sorted in Aemit and an index is built.
        So we make use of that.
        CrowCanyon new runs are full sky
    """
    edges = cat.attrs['aemitIndex.edges']
    offsets = cat.attrs['aemitIndex.offset']
    start, end = edges.searchsorted([amin, amax])
    if cat.comm.rank == 0:
        cat.logger.info("Range of index is %d to %d" %(( start + 1, end + 1)))
    start = offsets[start + 1]
    end = offsets[end + 1]

    cat =  cat.query_range(start, end)
    if cat.csize > 0:
        cat['RA'], cat['DEC'] = CartesianToEquatorial(cat['Position'], frame='galactic')
    else:
        cat['RA'] = 0
        cat['DEC'] = 0
    return cat